In [1]:
import os
import sys
import io
import json
import contextlib
from pathlib import Path
from dotenv import load_dotenv

root = Path().resolve()
while root != root.parent and not (root / 'backend').exists():
    root = root.parent
backend_root = root / 'backend'
if str(backend_root) not in sys.path:
    sys.path.insert(0, str(backend_root))

load_dotenv(backend_root / ".env")

# 로그 최소화
os.environ['RAG_LOG_RETRIEVER_DEBUG'] = '0'
os.environ['RAG_LOG_TIMING'] = '0'

from app.rag.tests.test_suite import GUIDE_TESTS
from app.rag.pipeline import RAGConfig, run_rag


In [2]:
results = []
for t in GUIDE_TESTS:
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        res = await run_rag(t['query'], config=RAGConfig(top_k=4, enable_consult_search=True))
    log = buf.getvalue().strip()
    results.append({"guidanceScript": res.get('guidanceScript', ''), "log": log})
print(json.dumps(results, ensure_ascii=False, indent=2))


[
  {
    "guidanceScript": "나라사랑카드를 분실하셔서 불편하시겠어요. 분실·도난 발생 즉시 해당 은행(카드사)로 신고하여, 금융 사고를 예방하셔야 합니다. 분실인지 도난인지 확인해 주세요",
    "log": ""
  },
  {
    "guidanceScript": "나라사랑카드를 분실하셨다니 걱정이 크시겠어요. 분실·도난 발생 즉시 해당 은행(카드사)로 신고하여, 금융 사고를 예방하셔야 합니다. 분실인지 도난인지 확인해 주세요",
    "log": ""
  },
  {
    "guidanceScript": "해외 여행 중 카드를 잃어버리셨다니 걱정이 크시겠어요. 카드의 분실·도난 시 신고 접수일로부터 60일 이전부터 발생하는 부정사용금액에 대해, 회원의 고의·과실이 있는 경우를 제외하고 보상받을 수 있습니다. 지금 카드 분실 신고를 진행하셨나요?",
    "log": ""
  },
  {
    "guidanceScript": "카드 도난으로 불편을 겪으신 점 안타깝습니다. 카드의 분실·도난 시 신고 접수일로부터 60일 이전부터 발생하는 부정사용금액에 대해, 회원의 고의·과실이 있는 경우를 제외하고 보상받을 수 있습니다. 지금 카드사에 신고하셨나요?",
    "log": ""
  },
  {
    "guidanceScript": "카드를 분실하셨다니 걱정이 크시겠어요. 카드의 분실·도난 시 신고 접수일로부터 60일 이전부터 발생하는 부정사용금액에 대해, 회원의 고의·과실이 있는 경우를 제외하고 보상받을 수 있습니다. 지금 카드사에 신고하실 준비가 되셨나요?",
    "log": ""
  },
  {
    "guidanceScript": "단기카드대출을 이용하시려는 상황에 대해 이해합니다. 신용카드의 상품별 이자율은 보통 연율로 표기되는데, 연율이란 1년을 단위로 하여 정한 이율이라는 뜻입니다. 단기카드대출 이용 시 신용점수에 영향을 줄 수 있다는 점을 알고 계셨나요?",
    "log": ""
  },
  {
   

In [3]:
results = []
for t in GUIDE_TESTS:
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        res = await run_rag(
            t['query'],
            config=RAGConfig(top_k=4, enable_consult_search=True, include_docs=True, include_consult_docs=True)
        )
    log = buf.getvalue().strip()
    docs = res.get('docs', []) or []
    consult_docs = res.get('consult_docs', []) or []

    top_docs = []
    for d in docs[:3]:
        top_docs.append({
            'id': d.get('id') or d.get('db_id'),
            'table': d.get('table'),
            'score': d.get('score'),
            'title': d.get('title') or (d.get('metadata') or {}).get('title'),
        })

    top_consult = []
    for d in consult_docs[:2]:
        top_consult.append({
            'id': d.get('id') or d.get('db_id'),
            'table': d.get('table'),
            'score': d.get('score'),
            'title': d.get('title') or (d.get('metadata') or {}).get('title'),
        })

    results.append({
        'id': t.get('id'),
        'query': t.get('query'),
        'guidanceScript': res.get('guidanceScript', ''),
        'top_docs': top_docs,
        'top_consult_docs': top_consult,
        'log': log,
    })

print(json.dumps(results, ensure_ascii=False, indent=2))


[
  {
    "id": "T001",
    "query": "나라사랑 잃어버렸어요",
    "guidanceScript": "나라사랑카드를 분실하셔서 불편하시겠어요. 분실·도난 발생 즉시 해당 은행(카드사)로 신고하여, 금융 사고를 예방하셔야 합니다. 분실인지 도난인지 확인해 주세요",
    "top_docs": [
      {
        "id": "narasarang_faq_005",
        "table": "service_guide_documents",
        "score": 2.0,
        "title": "나라사랑카드 분실ㆍ도난 시 어떻게 하나요?"
      },
      {
        "id": "narasarang_faq_006",
        "table": "service_guide_documents",
        "score": 1.0,
        "title": "나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?"
      }
    ],
    "top_consult_docs": [
      {
        "id": "hana_consultation_200483",
        "table": "consultation_documents",
        "score": 0.2006720184078187,
        "title": "도난/분실 신청/해제 상담"
      },
      {
        "id": "hana_consultation_26058",
        "table": "consultation_documents",
        "score": 0.188517179842669,
        "title": "도난/분실 신청/해제 상담"
      }
    ],
    "log": ""
  },
  {
    "id": "T002",
    "query": "나라사랑카드 분실하면 어떻게 해요",
    "guidanceScript": "나라사랑